In [15]:
import requests
import json
import getpass


## Connect to the musixmatch API to get lyrics:
You can sign up for a free developer account and generate a personal API key

https://developer.musixmatch.com/

In [12]:
api_key = getpass.getpass("api_key")

In [24]:
def get_songs(artist):
    api_url = "http://api.musixmatch.com/ws/1.1/track.search&apikey=" + api_key 
    url = api_url + "&q_artist=" + str(artist)
    x = requests.get(url)
    response = json.loads(x.text)

    return response['message']["body"]["track_list"]

In [33]:
# len("\n...\n\n******* This Lyrics is NOT for Commercial use *******") = 59

def get_lyrics(track_id):
    api_url = "http://api.musixmatch.com/ws/1.1/track.lyrics.get?apikey=" + api_key
    url = api_url + "&track_id="  + str(track_id)
    x = requests.get(url)
    response = json.loads(x.text)
    lyrics = response['message']["body"]["lyrics"]["lyrics_body"][:-59].split("\n")
    inner_hits = []
    for line in lyrics:
        if line not in ["", '...']:
            inner_hits.append({"line" : line})
    return inner_hits

In [34]:
songs = get_songs("hozier")

song_list = []
for song in songs:
    mysong = {}
    mysong["id"] = song["track"]["track_id"]
    mysong["name"] = song["track"]["track_name"].split("(")[0].split("-")[0]
    mysong["artist"] = song["track"]["artist_name"]
    if (song["track"]["has_lyrics"] == 1):
        mysong["lyrics"] = get_lyrics(mysong["id"])
    else:
        mysong["lyrics"] = None
    song_list.append(mysong)

In [36]:
with open('data/hozier_songs.json', 'w') as f:
    json.dump(song_list, f)

## Pydantic Model example for validating data input

In [37]:
import json
with open('data/hozier_songs.json', 'r') as f:
  songs = json.load(f)

In [38]:
song = songs[0]

In [40]:
from pydantic import BaseModel

class Song(BaseModel):
    name: str
    artist: str
    lyrics: list | None
    id : int

    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Take me to church",
                    "artist": "Hozier",
                    "lyrics": ["Her eyes and words are so icy", "Oh but she burns", "Like rum on the fire"],
                    "id": 0,
                }
            ]
        }
    }
    
example = Song.model_validate(song)

In [41]:
example

Song(name='Cherry Wine ', artist='Hozier', lyrics=[{'line': 'Her eyes and words are so icy'}, {'line': 'Oh but she burns'}, {'line': 'Like rum on the fire'}, {'line': 'Hot and fast and angry as she can be'}, {'line': 'I walk my days on a wire.'}, {'line': "It looks ugly, but it's clean,"}, {'line': "Oh momma, don't fuss over me."}, {'line': "The way she tells me I'm hers and she is mine"}, {'line': 'Open hand or closed fist would be fine'}, {'line': 'The blood is rare and sweet as cherry wine.'}, {'line': 'Calls of guilty thrown at me'}, {'line': '******* Th'}], id=84741219)

In [42]:
example.lyrics

[{'line': 'Her eyes and words are so icy'},
 {'line': 'Oh but she burns'},
 {'line': 'Like rum on the fire'},
 {'line': 'Hot and fast and angry as she can be'},
 {'line': 'I walk my days on a wire.'},
 {'line': "It looks ugly, but it's clean,"},
 {'line': "Oh momma, don't fuss over me."},
 {'line': "The way she tells me I'm hers and she is mine"},
 {'line': 'Open hand or closed fist would be fine'},
 {'line': 'The blood is rare and sweet as cherry wine.'},
 {'line': 'Calls of guilty thrown at me'},
 {'line': '******* Th'}]